In [ ]:
import cobra
from cobra.io import read_sbml_model
from pprint import pprint
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from cobra.medium import minimal_medium

from process_bigraph import Composite
from process_bigraph import ProcessTypes
from process_bigraph.emitter import gather_emitter_results

from cdFBA import register_types
from cdFBA.processes.dfba import dFBA, UpdateEnvironment, StaticConcentration, Injector, WaveFunction

from cdFBA.utils import make_cdfba_composite, get_injector_spec, get_wave_spec, get_static_spec, set_concentration, set_kinetics, get_exchanges

In [ ]:
# Goal: Replicate Adrian et al. model quantitatively using cdFBA

# 3 bacteria species
# /Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Bacteroides_thetaiotaomicron_VPI_5482.xml
# /Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Methanobrevibacter_smithii_ATCC_35061.xml
# /Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Eubacterium_rectale_ATCC_33656.xml

In [ ]:
b_thetaiotaomicron = cobra.io.read_sbml_model(
    "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Bacteroides_thetaiotaomicron_VPI_5482.xml")
b_thetaiotaomicron.optimize()
b_thetaiotaomicron.summary()

In [ ]:
e_rectale = cobra.io.read_sbml_model("/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Eubacterium_rectale_ATCC_33656.xml")
# e_rectale.optimize()
# e_rectale.summary()

In [ ]:
m_simithii = cobra.io.read_sbml_model("/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Methanobrevibacter_smithii_ATCC_35061.xml")
# m_simithii.optimize()
# m_simithii.summary()

In [ ]:
# interested in following exchange reactions: acetate, butyrate, CO2, H2, CH4, and a polysaccharide

# make it so that we are in minimal media (have NO other carbon source besides cholate)

In [ ]:
gut_models = {
        "B_thetaiotaomicron": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Bacteroides_thetaiotaomicron_VPI_5482.xml",
        "E_rectale": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Eubacterium_rectale_ATCC_33656.xml",
        "Methanobrevibacter_smithii": "/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Methanobrevibacter_smithii_ATCC_35061.xml"
    }

In [ ]:
models = {k: read_sbml_model(v) for k, v in gut_models.items()}

In [ ]:
# get list of exchange reactions for all three species - combine into a set
# do one species at a time
# get exchanges funciton

In [ ]:
# # for minimal media
# bacteroides_mm = get_exchanges("/Users/rebekahsheih/PycharmProjects/Rebekahs_Rotation_Project/Vivarium/cdFBA-main/Notebooks/sbml/Bacteroides_thetaiotaomicron_VPI_5482.xml", medium_type = "minimal")
#
# pprint(bacteroides_mm)
#
# exchanges = ['EX_ac(e)', 'EX_but(e)', 'EX_co2(e)', 'EX_h(e)', 'EX_ch4(e)', 'EX_cholate(e)'] # acetate, butyrate, CO2, H2, methane, exopolysaccharide, Google how to combine 3 exchange reaction lists into a set

In [ ]:
exchanges = ['EX_ac(e)', 'EX_but(e)', 'EX_ch4(e)', 'EX_pect(e)'] # acetate, butyrate, CO2, H2, methane, pectins, Google how to combine 3 exchange

volume = 2

# dFBA model
spec = make_cdfba_composite(gut_models, medium_type=None, exchanges=exchanges, volume=volume, interval=0.1)

pprint(spec)

In [ ]:
#Set reaction bounds
spec['Species']['B_thetaiotaomicron']['config']['bounds'] = {
            "EX_o2(e)": {"lower": -2, "upper": None},
            "DM_atp_c_": {"lower": 1, "upper": 1} # cut off carbon source
        }
spec['Species']['E_rectale']['config']['bounds'] = {
            "EX_o2(e)": {"lower": -2, "upper": None},
            "DM_atp_c_": {"lower": 1, "upper": 1} # cut off carbon source
        }
spec['Species']['Methanobrevibacter_smithii']['config']['bounds'] = {
            "EX_o2(e)": {"lower": -2, "upper": None},
            "DM_atp_c_": {"lower": 1, "upper": 1} # cut off carbon source
        }

In [ ]:
#set external substrate concentrations
concentrations = {      # abundance of polysaccharide, limit acetate so that it has to come from keystone species
    'acetate': 0,
    'pectins': 5,
    'butyrate': 0,
    'Methane': 0
}
set_concentration(spec, concentrations)

In [ ]:
#set kinetics
kinetics = {
    'acetate': (0.5, 5),
    'pectins': (0.2, 7), # reduce km, increase Vmax
    'butyrate': (0.5, 5),
    'Methane': (0.5, 5),
}
for species in gut_models.keys():
    set_kinetics(species, spec, kinetics)
pprint(spec)

In [ ]:
#set emitter specs
spec['emitter'] = {
        "_type": "step",
        "address": "local:ram-emitter",
        "config": {
            "emit": {
                "shared_environment": "any",
                "global_time": "any",
            }
        },
        "inputs": {
            "shared_environment": ["Shared Environment"],
            "global_time": ["global_time"]
        }
    }

In [ ]:
#create the core object
core = ProcessTypes()
#register data types
core = register_types(core)
#register all processes and steps
core.register_process('dFBA', dFBA)
core.register_process('UpdateEnvironment', UpdateEnvironment)
core.register_process('StaticConcentration', StaticConcentration)
core.register_process('WaveFunction', WaveFunction)
core.register_process('Injector', Injector)

In [ ]:
#create simulation composite
sim = Composite({
        "state": spec,
        },
        core=core
    )

In [ ]:
#run simulation
sim.run(20)

In [ ]:
#gather results
results = gather_emitter_results(sim)[('emitter',)]

In [ ]:
#extract time-series data
timepoints = []
for timepoint in results:
    time = timepoint.pop('global_time')
    timepoints.append(time)
env = [timepoint['shared_environment']['concentrations'] for timepoint in results]
env_combined = {}
for d in env:
    for key, value in d.items():
        if key not in env_combined:
            env_combined[key] = []
        env_combined[key].append(value)

In [ ]:
results

In [ ]:
#plot results for biomass
fig, ax = plt.subplots(dpi=300)
for key, value in env_combined.items():
    if key not in ['acetate', 'pectins', 'carbon dioxide', 'butyrate', 'Methane', 'proton']:
        ax.plot(timepoints, env_combined[key], label=key)
        ax.set_yscale('log')
plt.xlabel('Time (h)')
plt.ylabel('Biomass (gDW)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#plot substrates
fig, ax = plt.subplots(dpi=300)
for key, value in env_combined.items():
    if key in ['acetate', 'pectins', 'butyrate', 'Methane']:
        ax.plot(timepoints, env_combined[key], label=key)
        ax.set_yscale('log')
plt.xlabel('Time (h)')
plt.ylabel('Substrate Concentration (mM)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# TODO
# Look at just ONE species and see if it's consuming polysaccharide and producing acetate
# Try different polysaccharides and see if a different one is consumed more